In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords

## Intro to word2vec

The most common unsupervised neural network approach for NLP is word2vec, a shallow neural network model for converting words to vectors using distributed representation: Each word is represented by many neurons, and each neuron is involved in representing many words.  At the highest level of abstraction, word2vec assigns a vector of random values to each word.  For a word W, it looks at the words that are near W in the sentence, and shifts the values in the word vectors such that the vectors for words near that W are closer to the W vector, and vectors for words not near W are farther away from the W vector.  With a large enough corpus, this will eventually result in words that often appear together having vectors that are near one another, and words that rarely or never appear together having vectors that are far away from each other.  Then, using the vectors, similarity scores can be computed for each pair of words by taking the cosine of the vectors.  

This may sound quite similar to the Latent Semantic Analysis approach you just learned.  The conceptual difference is that LSA creates vector representations of sentences based on the words in them, while word2vec creates representations of individual words, based on the words around them.

## What is it good for?

Word2vec is useful for any time when computers need to parse requests written by humans. The problem with human communication is that there are so many different ways to communicate the same concept. It's easy for us, as humans, to know that "the silverware" and "the utensils" can refer to the same thing. Computers can't do that unless we teach them, and this can be a real chokepoint for human/computer interactions. If you've ever played a text adventure game (think _Colossal Cave Adventure_ or _Zork_), you may have encountered the following scenario:

And your brain explodes from frustration. A text adventure game that incorporates a properly trained word2vec model would have vectors for "pick up", "lift", and "take" that are close to the vector for "grab" and therefore could accept those other verbs as synonyms so you could move ahead faster. In more practical applications, word2vec and other similar algorithms are what help a search engine return the best results for your query and not just the ones that contain the exact words you used. In fact, search is a better example, because not only does the search engine need to understand your request, it also needs to match it to web pages that were _also written by humans_ and therefore _also use idiosyncratic language_.

Humans, man.  

So how does it work?

## Generating vectors: Multiple algorithms

In considering the relationship between a word and its surrounding words, word2vec has two options that are the inverse of one another:

 * _Continuous Bag of Words_ (CBOW): the identity of a word is predicted using the words near it in a sentence.
 * _Skip-gram_: The identities of words are predicted from the word they surround. Skip-gram seems to work better for larger corpuses.

For the sentence "Terry Gilliam is a better comedian than a director", if we focus on the word "comedian" then CBOW will try to predict "comedian" using "is", "a", "better", "than", "a", and "director".  Skip-gram will try to predict "is", "a", "better", "than", "a", and "director" using the word "comedian". In practice, for CBOW the vector for "comedian" will be pulled closer to the other words, while for skip-gram the vectors for the other words will be pulled closer to "comedian".  

In addition to moving the vectors for nearby words closer together, each time a word is processed some vectors are moved farther away. Word2vec has two approaches to "pushing" vectors apart:
 
 * _Negative sampling_: Like it says on the tin, each time a word is pulled toward some neighbors, the vectors for a randomly chosen small set of other words are pushed away.
 * _Hierarchical softmax_: Every neighboring word is pulled closer or farther from a subset of words chosen based on a tree of probabilities.

## What is similarity? Word2vec strengths and weaknesses

Keep in mind that word2vec operates on the assumption that frequent proximity indicates similarity, but words can be "similar" in various ways. They may be conceptually similar ("royal", "king", and "throne"), but they may also be functionally similar ("tremendous" and "negligible" are both common modifiers of "size"). Here is a more detailed exploration, [with examples](https://quomodocumque.wordpress.com/2016/01/15/messing-around-with-word2vec/), of what "similarity" means in word2vec.

One cool thing about word2vec is that it can identify similarities between words _that never occur near one another in the corpus_. For example, consider these sentences:

"The dog played with an elastic ball."
"Babies prefer the ball that is bouncy."
"I wanted to find a ball that's elastic."
"Tracy threw a bouncy ball."

"Elastic" and "bouncy" are similar in meaning in the text but don't appear in the same sentence. However, both appear near "ball". In the process of nudging the vectors around so that "elastic" and "bouncy" are both near the vector for "ball", the words also become nearer to one another and their similarity can be detected.

For a while after it was introduced, [no one was really sure why word2vec worked as well as it did](https://arxiv.org/pdf/1402.3722v1.pdf) (see last paragraph of the linked paper). A few years later, some additional math was developed to explain word2vec and similar models. If you are comfortable with both math and "academese", have a lot of time on your hands, and want to take a deep dive into the inner workings of word2vec, [check out this paper](https://arxiv.org/pdf/1502.03520v7.pdf) from 2016.  

One of the draws of word2vec when it first came out was that the vectors could be used to convert analogies ("king" is to "queen" as "man" is to "woman", for example) into mathematical expressions ("king" + "woman" - "man" = ?) and solve for the missing element ("queen"). This is kinda nifty.

A drawback of word2vec is that it works best with a corpus that is at least several billion words long. Even though the word2vec algorithm is speedy, this is a a lot of data and takes a long time! Our example dataset is only two million words long, which allows us to run it in the notebook without overwhelming the kernel, but probably won't give great results.  Still, let's try it!

There are a few word2vec implementations in Python, but the general consensus is the easiest one to us is in [gensim](https://radimrehurek.com/gensim/models/word2vec.html). Now is a good time to `pip install gensim` if you don't have it yet.

In [2]:
# Utility function to clean text.
def text_cleaner(text):
    
    # Visual inspection shows spaCy does not recognize the double dash '--'.
    # Better get rid of it now!
    text = re.sub(r'--',' ',text)
    
    # Get rid of headings in square brackets.
    text = re.sub("[\[].*?[\]]", "", text)
    
    # Get rid of chapter titles.
    text = re.sub(r'Chapter \d+','',text)
    
    # Get rid of extra whitespace.
    text = ' '.join(text.split())
    
    return text


# Import all the Austen in the Project Gutenberg corpus.
austen = ""
for novel in ['persuasion','emma','sense']:
    work = gutenberg.raw('austen-' + novel + '.txt')
    austen = austen + work

# Clean the data.
austen_clean = text_cleaner(austen)

In [3]:
austen_clean_p1 = austen_clean[0:900000]

In [4]:
austen_clean_p2 = austen_clean[900000:1800000]

In [5]:
austen_clean_p3 = austen_clean[1800000:2006273]

In [6]:
# Parse the data. This can take some time.
nlp = spacy.load('en')
austen_doc_p1 = nlp(austen_clean_p1)
austen_doc_p2 = nlp(austen_clean_p2)
austen_doc_p3 = nlp(austen_clean_p3)

In [7]:
# Organize the parsed doc into sentences, while filtering out punctuation
# and stop words, and converting words to lower case lemmas.
sentences_p1 = []
for sentence in austen_doc_p1.sents:
    sentence = [
        token.lemma_.lower()
        for token in sentence
        if not token.is_stop
        and not token.is_punct
    ]
    sentences_p1.append(sentence)


print(sentences_p1[20])
print('We have {} sentences and {} tokens.'.format(len(sentences_p1), len(austen_clean_p1)))

['for', 'daughter', 'eld', 'give', 'thing', 'tempt']
We have 7771 sentences and 900000 tokens.


In [8]:
# Organize the parsed doc into sentences, while filtering out punctuation
# and stop words, and converting words to lower case lemmas.
sentences_p2 = []
for sentence in austen_doc_p2.sents:
    sentence = [
        token.lemma_.lower()
        for token in sentence
        if not token.is_stop
        and not token.is_punct
    ]
    sentences_p2.append(sentence)


print(sentences_p2[20])
print('We have {} sentences and {} tokens.'.format(len(sentences_p2), len(austen_clean_p2)))

['yes', 'say', 'jane', 'hear', 'kind', 'offer', 'hear', 'thing']
We have 8577 sentences and 900000 tokens.


In [9]:
# Organize the parsed doc into sentences, while filtering out punctuation
# and stop words, and converting words to lower case lemmas.
sentences_p3 = []
for sentence in austen_doc_p3.sents:
    sentence = [
        token.lemma_.lower()
        for token in sentence
        if not token.is_stop
        and not token.is_punct
    ]
    sentences_p3.append(sentence)


print(sentences_p3[20])
print('We have {} sentences and {} tokens.'.format(len(sentences_p3), len(austen_clean_p3)))

['and', 'thing', 'prepare', 'bad', 'mother', 'determine', 'natural', 'kind', 'spirit', 'settle', 'that', 'estate', 'robert', 'immediately', 'edward', "'s", 'proper', 'condition']
We have 1506 sentences and 206272 tokens.


In [10]:
sentences = sentences_p1 + sentences_p2 + sentences_p3

In [11]:
import gensim
from gensim.models import word2vec

model = word2vec.Word2Vec(
    sentences,
    workers=4,     # Number of threads to run in parallel (if your computer does parallel processing).
    min_count=10,  # Minimum word count threshold.
    window=6,      # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=1e-3 ,  # Penalize frequent words.
    size=300,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)

print('done!')

done!


In [12]:
# List of words in model.
vocab = model.wv.vocab.keys()

print(model.wv.most_similar(positive=['lady', 'man'], negative=['woman']))

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model.wv.similarity('loud', 'aloud'))
print(model.wv.similarity('mr', 'mrs'))

# One of these things is not like the other...
print(model.doesnt_match("breakfast marriage dinner lunch".split()))

[('people', 0.678819477558136), ('cordial', 0.5522921681404114), ('person', 0.4514036476612091), ('thousand', 0.44400525093078613), ('discretion', 0.43628740310668945), ('nineteen', 0.42778998613357544), ('captain', 0.42705827951431274), ('ought', 0.42069676518440247), ('introduction', 0.41460198163986206), ('fellow', 0.4132372736930847)]
0.72508484
0.23594444
marriage


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  if sys.path[0] == '':


In [13]:
vocab

dict_keys(['sir', 'walter', 'elliot', 'kellynch', 'hall', 'somersetshire', 'man', 'amusement', 'take', 'book', 'find', 'occupation', 'idle', 'hour', 'consolation', 'distressed', 'rouse', 'admiration', 'respect', 'early', 'unwelcome', 'sensation', 'arise', 'domestic', 'affair', 'change', 'naturally', 'pity', 'contempt', 'turn', 'read', 'history', 'interest', 'fail', 'this', 'favourite', 'volume', 'open', 'of', 'bear', 'married', 'elizabeth', 'daughter', 'james', 'south', 'park', 'county', 'lady', 'die', 'june', 'anne', 'son', 'november', 'mary', 'precisely', 'stand', "'s", 'hand', 'improve', 'add', 'information', 'family', 'word', 'date', 'birth', 'charles', 'heir', 'musgrove', 'uppercross', 'day', 'month', 'lose', 'wife', 'then', 'follow', 'rise', 'respectable', 'usual', 'term', 'settle', 'mention', 'serve', 'office', 'high', 'represent', 'exertion', 'dignity', 'baronet', 'year', 'marry', 'form', 'altogether', 'handsome', 'conclude', 'arm', 'principal', 'seat', 'william', 'great', 'sec

Clearly this model is not great – while some words given above might possibly fill in the analogy woman:lady::man:?, most answers likely make little sense. You'll notice as well that re-running the model likely gives you different results, indicating random chance plays a large role here.

We do, however, get a nice result on "marriage" being dissimilar to "breakfast", "lunch", and "dinner". 

## Drill 0

Take a few minutes to modify the hyperparameters of this model and see how its answers change. Can you wrangle any improvements?

### Iteration

In [14]:
# Tinker with hyperparameters here.

model = word2vec.Word2Vec(
    sentences,
    workers=4,     # Number of threads to run in parallel (if your computer does parallel processing).
    min_count=10,  # Minimum word count threshold.
    window=6,      # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=1e-4 ,  # Penalize frequent words.
    size=300,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)

# List of words in model.
vocab = model.wv.vocab.keys()

print(model.wv.most_similar(positive=['lady', 'man'], negative=['woman']))
print('\n')

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model.wv.similarity('loud', 'aloud'))
print(model.wv.similarity('mr', 'mrs'))

# One of these things is not like the other...
print(model.doesnt_match("breakfast marriage dinner lunch".split()))

[('inconvenient', 0.9985440969467163), ('discretion', 0.9984819889068604), ('opposite', 0.9983533620834351), ('possibility', 0.9981647729873657), ('parting', 0.9980906844139099), ('wait', 0.9980711340904236), ('read', 0.9980366826057434), ('box', 0.9980331659317017), ('calmly', 0.9980219006538391), ('indulge', 0.9979931116104126)]


0.9960654
0.99388075
breakfast


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).


### Iteration

In [15]:
# Tinker with hyperparameters here.

model = word2vec.Word2Vec(
    sentences,
    workers=4,     # Number of threads to run in parallel (if your computer does parallel processing).
    min_count=10,  # Minimum word count threshold.
    window=6,      # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=5e-3 ,  # Penalize frequent words.
    size=300,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)

# List of words in model.
vocab = model.wv.vocab.keys()

print(model.wv.most_similar(positive=['lady', 'man'], negative=['woman']))
print('\n')

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model.wv.similarity('loud', 'aloud'))
print(model.wv.similarity('mr', 'mrs'))

# One of these things is not like the other...
print(model.doesnt_match("breakfast marriage dinner lunch".split()))

[('farmer', 0.5028352737426758), ('people', 0.4907006025314331), ('husband', 0.4715992212295532), ('spoil', 0.4458686113357544), ('rapid', 0.4309927225112915), ('visit', 0.4178653061389923), ('absolute', 0.41309207677841187), ('parent', 0.40888017416000366), ('joke', 0.39972272515296936), ('friend', 0.39867907762527466)]


0.6800617
0.3546172
marriage


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).


### Iteration

In [16]:
# Tinker with hyperparameters here.

model = word2vec.Word2Vec(
    sentences,
    workers=4,     # Number of threads to run in parallel (if your computer does parallel processing).
    min_count=10,  # Minimum word count threshold.
    window=8,      # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=1e-3 ,  # Penalize frequent words.
    size=300,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)

# List of words in model.
vocab = model.wv.vocab.keys()

print(model.wv.most_similar(positive=['lady', 'man'], negative=['woman']))
print('\n')

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model.wv.similarity('loud', 'aloud'))
print(model.wv.similarity('mr', 'mrs'))

# One of these things is not like the other...
print(model.doesnt_match("breakfast marriage dinner lunch".split()))

[('people', 0.523370623588562), ('cordial', 0.4126809239387512), ('fine', 0.398892879486084), ('monstrous', 0.38834452629089355), ('abroad', 0.3863050937652588), ('dreadful', 0.37943828105926514), ('way', 0.37533122301101685), ('ought', 0.3673796057701111), ('winthrop', 0.3664364218711853), ('anne', 0.3635774254798889)]


0.7610396
0.18682437
marriage


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).


### Iteration

In [17]:
# Tinker with hyperparameters here.

model = word2vec.Word2Vec(
    sentences,
    workers=4,     # Number of threads to run in parallel (if your computer does parallel processing).
    min_count=10,  # Minimum word count threshold.
    window=6,      # Number of words around target word to consider.
    sg=1,          # Use CBOW because our corpus is small.
    sample=1e-3 ,  # Penalize frequent words.
    size=300,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)

# List of words in model.
vocab = model.wv.vocab.keys()

print(model.wv.most_similar(positive=['lady', 'man'], negative=['woman']))
print('\n')

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model.wv.similarity('loud', 'aloud'))
print(model.wv.similarity('mr', 'mrs'))

# One of these things is not like the other...
print(model.doesnt_match("breakfast marriage dinner lunch".split()))

[('meet', 0.5692358016967773), ('please', 0.563198447227478), ('kellynch', 0.5623747706413269), ('entertain', 0.5503693222999573), ('worth', 0.539549708366394), ('polite', 0.5383217334747314), ('hall', 0.5360033512115479), ('russell', 0.5262104272842407), ('introduce', 0.5252732038497925), ('drop', 0.5157787799835205)]


0.69841826
0.56567967
marriage


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).


### Iteration

In [18]:
# Tinker with hyperparameters here.

model = word2vec.Word2Vec(
    sentences,
    workers=4,     # Number of threads to run in parallel (if your computer does parallel processing).
    min_count=10,  # Minimum word count threshold.
    window=6,      # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=1e-3 ,  # Penalize frequent words.
    size=300,      # Word vector length.
    hs=0           # Use hierarchical softmax.
)

# List of words in model.
vocab = model.wv.vocab.keys()

print(model.wv.most_similar(positive=['lady', 'man'], negative=['woman']))
print('\n')

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model.wv.similarity('loud', 'aloud'))
print(model.wv.similarity('mr', 'mrs'))

# One of these things is not like the other...
print(model.doesnt_match("breakfast marriage dinner lunch".split()))

[('mother', 0.979598879814148), ('year', 0.9794370532035828), ('family', 0.9793519377708435), ('give', 0.9793475270271301), ('feeling', 0.9791440963745117), ('house', 0.9790881276130676), ('place', 0.9786862134933472), ('people', 0.9786702990531921), ('mind', 0.9785785675048828), ('old', 0.9783843755722046)]


0.9991087
0.9917786
marriage


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).


## Drill 0 Conclusions

Both negative sampling and a lower "sample" parameter led to overinflated similarities.

# Example word2vec applications

You can use the vectors from word2vec as features in other models, or try to gain insight from the vector compositions themselves.

Here are some neat things people have done with word2vec:

 * [Visualizing word embeddings in Jane Austen's Pride and Prejudice](http://blogger.ghostweather.com/2014/11/visualizing-word-embeddings-in-pride.html). Skip to the bottom to see a _truly honest_ account of this data scientist's process.

 * [Tracking changes in Dutch Newspapers' associations with words like 'propaganda' and 'alien' from 1950 to 1990](https://www.slideshare.net/MelvinWevers/concepts-through-time-tracing-concepts-in-dutch-newspaper-discourse-using-sequential-word-vector-spaces).

 * [Helping customers find clothing items similar to a given item but differing on one or more characteristics](http://multithreaded.stitchfix.com/blog/2015/03/11/word-is-worth-a-thousand-vectors/).

## Drill 1: Word2Vec on 100B+ words

As we mentioned, word2vec really works best on a big corpus, but it can take half a day to clean such a corpus and run word2vec on it.  Fortunately, there are word2vec models available that have already been trained on _really_ big corpora. They are big files, but you can download a [pretrained model of your choice here](https://github.com/3Top/word2vec-api). At minimum, the ones built with word2vec (check the "Architecture" column) should load smoothly using an appropriately modified version of the code below, and you can play to your heart's content.

Because the models are so large, however, you may run into memory problems or crash the kernel. If you can't get a pretrained model to run locally, check out this [interactive web app of the Google News model](https://rare-technologies.com/word2vec-tutorial/#bonus_app) instead.

However you access it, play around with a pretrained model. Is there anything interesting you're able to pull out about analogies, similar words, or words that don't match? Write up a quick note about your tinkering and discuss it with your mentor during your next session.

In [19]:
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format ('GoogleNews-vectors-negative300.bin', binary=True)

In [20]:
# Play around with your pretrained model here.

# List of words in model.
vocab = model.vocab.keys()


In [21]:
len(vocab)

3000000

In [43]:
'bicycle' in vocab

True

In [45]:
model.vocab['bicycle']

In [46]:
model.vocab['demon']

In [22]:

print(model.most_similar(positive=['lady', 'man'], negative=['woman']))
print('\n')

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model.similarity('loud', 'aloud'))
print(model.similarity('mr', 'mrs'))

# One of these things is not like the other...
print(model.doesnt_match("breakfast marriage dinner lunch".split()))


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('fella', 0.6031545400619507), ('gentleman', 0.5849649906158447), ('chap', 0.5543248653411865), ('gent', 0.543907880783081), ('guy', 0.5265033841133118), ('lad', 0.5139425992965698), ('feller', 0.5072450041770935), ('bloke', 0.49030160903930664), ('rascal', 0.4873698949813843), ('ladies', 0.47617611289024353)]


0.38398623
0.66098833
marriage


The pretrained model is able to complete the analogy correctly, with terms that correspond to alternate ways of referring to men. However, many of these words lack the appropriate formality of "gentleman" or "gent,' perhaps because of common modern-day casual usage of "lady." At least "germ" wasn't in the top 10.

In [23]:
print(model.similarity('cat', 'dog'))
print(model.similarity('cat', 'lizard'))
print(model.similarity('cat', 'cow'))
print(model.similarity('cat', 'ant'))
print('\n')
print(model.similarity('cat', 'hammer'))
print(model.similarity('cat', 'bagel'))
print(model.similarity('cat', 'dignity'))
print('\n')
print(model.similarity('cat', 'spindle'))
print(model.similarity('cat', 'ergonomic'))
print(model.similarity('cat', 'axiom'))
print(model.similarity('cat', 'relinquish'))
print('\n')
print(model.similarity('cat', 'fuzzy'))
print(model.similarity('cat', 'cradle'))
print(model.similarity('cat', 'claws'))
print(model.similarity('cat', 'pajamas'))

0.76094574
0.5207694
0.4035246
0.2832825


0.09537769
0.24715711
0.052280243


0.03710579
0.054001525
0.09184816
0.080776885


0.2567619
0.075048946
0.3623707
0.2624267


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [24]:
print(model.similarity('break', 'pause'))
print(model.similarity('break', 'intermission'))
print(model.similarity('break', 'rest'))
print(model.similarity('break', 'interval'))
print(model.similarity('break', 'lull'))
print(model.similarity('break', 'respite'))
print('\n')
print(model.similarity('break', 'escape'))
print('\n')
print(model.similarity('break', 'smash'))
print(model.similarity('break', 'shatter'))
print(model.similarity('break', 'destroy'))
print(model.similarity('break', 'crack'))
print(model.similarity('break', 'fracture'))
print('\n')
print(model.similarity('brake', 'stop'))



0.2717482
0.27857122
0.24203287
0.31065407
0.34142017
0.30789012


0.3440333


0.4046733
0.4530429
0.26714057
0.44822454
0.21647625


0.27012914


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [25]:
print(model.most_similar(positive=['pants', 'shirt'], negative=['torso']))
print('\n')

/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('shirts', 0.6786606907844543), ('t_shirt', 0.6128919720649719), ('T_shirt', 0.6011109352111816), ('jacket', 0.5979953408241272), ('blazer', 0.593267560005188), ('sweatshirt', 0.5822854042053223), ('sweater', 0.5800738334655762), ('jeans', 0.5789099931716919), ('trousers', 0.5755550861358643), ('polo_shirts', 0.5674843788146973)]




In [26]:
print(model.most_similar(positive=['think', 'run'], negative=['thought']))
print('\n')

/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('runs', 0.5420483946800232), ('running', 0.5055066347122192), ('go', 0.44135770201683044), ('walk', 0.39560920000076294), ('do', 0.3941177427768707), ('drive', 0.39071887731552124), ('Mark_Grudzielanek_singled', 0.3755759596824646), ('surpass_Hank_Aaron', 0.37111562490463257), ('operate', 0.3666856288909912), ('homerun', 0.3659612834453583)]




In [27]:
# One of these things is not like the other...
print(model.doesnt_match("rain snow applesauce sleet".split()))

applesauce


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [28]:
# One of these things is not like the other...
print(model.doesnt_match("electron proton neutron potato".split()))

potato


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [29]:
# One of these things is not like the other...
print(model.doesnt_match("break shatter destroy smash".split()))

destroy


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [30]:
# One of these things is not like the other...
print(model.doesnt_match("break pause hiatus".split()))

break


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


## Extension: Working with filetype other than ".bin"

In [31]:
model = gensim.models.KeyedVectors.load_word2vec_format ('glove.twitter.27B/glove.twitter.27B.200d.txt', binary=True)

FileNotFoundError: [Errno 2] No such file or directory: 'glove.twitter.27B/glove.twitter.27B.200d.txt'